# Настройка среды

In [1]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [2]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.losses import MseCrossEntropyCombination
from elements.abstracts import AbstractModulator
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from elements.simple import AdjustSize, AmplitudeToIntensity, IntensityToAmplituder, Empty
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
import torch
import torchvision
import numpy
import timm
import pandas
import json
from copy import deepcopy
from cluster import train, confusion, execute, epochs, SelectedGPUs
from PIL import Image
from itertools import product
import optuna

In [3]:
FontLibrary.Fonts.PlotTitle.FontSize = 16

FontLibrary.Fonts.DescriptionLeft.FontSize = 12
FontLibrary.Fonts.DescriptionLeft.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionLeft.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionBottom.FontSize = 10
FontLibrary.Fonts.DescriptionBottom.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionBottom.FontStyle = 'italic'

FontLibrary.Fonts.DescriptionTop.FontSize = 10
FontLibrary.Fonts.DescriptionTop.FontWeight = 'bold'
FontLibrary.Fonts.DescriptionTop.FontStyle = 'italic'

FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FontLibrary.Fonts.AxisX.FontSize = 8
FontLibrary.Fonts.AxisY.FontSize = 8

FontLibrary.Fonts.GraphTitle.FontSize = 10

FigureWidthHeight = (6.69291, 10.1181-1.0)

In [4]:
device = torch.device(2 if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: NVIDIA A100-SXM4-80GB


# Определение методов

In [5]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False, in_chans=1, num_classes=classes).to(device)

# Определение цели для Optuna

In [6]:
def objective(trial:optuna.trial):
    # Постоянные параметры 
    N = 1536//6 #1536
    pixels = 1536//6 #1536
    length = 3.0E-3
    wavelength = 500.0E-9
    spatial_coherence = 100.0E-6
    time_coherence = 10.0E-9
    time_relaxation = 1.0E-6
    mean_samples = 20 #20
    detectors_amount =/ 24
    detector_size = length / 60 

    # Записываем постоянные параметры
    trial.set_user_attr('N',                 N)
    trial.set_user_attr('pixels',            pixels)
    trial.set_user_attr('length',            length)
    trial.set_user_attr('wavelength',        wavelength)
    trial.set_user_attr('spatial_coherence', spatial_coherence)
    trial.set_user_attr('time_coherence',    time_coherence)
    trial.set_user_attr('time_relaxation',   time_relaxation)
    trial.set_user_attr('mean_samples',      mean_samples)
    trial.set_user_attr('detectors',         detectors_amount)
    trial.set_user_attr('detectors_size',    detector_size)
    
    # Варьируемые параметры
    masks_amount              = trial.suggest_int('masks amount',               3,      10)
    batch_size                = trial.suggest_int('batch size',                 32//8,  40//8)
    distance                  = trial.suggest_float('distance',                 5.0E-3, 500.0E-3)
    learning_rate             = trial.suggest_float('learning rate',            1.0E-6, 1.0E-2)
    loss_functions_proportion = trial.suggest_float('cross entorpy proportion', 0.0,    1.0)
    optimizer_type            = trial.suggest_categorical('optimizer type',     ["Adam", "RMSprop"])
    optimizer_type = {"Adam":torch.optim.Adam, "RMSprop":torch.optim.RMSprop}[optimizer_type]
    loss_function = MseCrossEntropyCombination(loss_functions_proportion)
    
    # Определение датасета
    dataset = Dataset('CIFAR10', batch_size, N, N, torch.float32, threads=8, preload=10)
    # dataset.padding(surface_ratio=0.3)
    dataset.load.train()
    dataset.load.test()

    # Модуль некогерентности
    incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length)
    incoherent_encoder, incoherent_decoder = incoherent.pair()
    incoherent_encoder, incoherent_decoder = incoherent_encoder.to(device), incoherent_decoder.to(device)
    incoherent.sample()
    incoherent_encoder, incoherent_decoder = incoherent_encoder.cpu(), incoherent_decoder.cpu()
    torch.cuda.empty_cache()

    # Инициализация детекторов
    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((detector_size, detector_size), (0,0))
    detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter)

    # Инициализация дополнительных можулей
    int_to_amp = IntensityToAmplituder()
    full_propagation =  FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance*masks_amount, 0.4)
    
    # Инициализация электронной, оптической части и гибридной модели
    electronic = timm.create_model('resnet18', pretrained=False,  in_chans=1, num_classes=10)
    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators     = [PhaseModulator(N, length, pixels)     for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [phase_modulators[0], amplitude_modulators[0]]
    for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
        elements.append(propagation)
        elements.append(phase_modulator)
        elements.append(amplitude_modulator)
    elements.append(propagation)
    optical = CompositeModel(int_to_amp, incoherent_encoder, *elements, incoherent_decoder)
    model = HybridModel(optical, detectors, electronic)

    # Инициализация пустой модели (без оптической части)
    empty_incoherent = deepcopy(incoherent)
    empty_incoherent_encoder, empty_incoherent_decoder = empty_incoherent.pair()
    empty_optical = CompositeModel(int_to_amp, empty_incoherent_encoder, full_propagation, empty_incoherent_decoder)
    empty_model = HybridModel(empty_optical, deepcopy(detectors), deepcopy(electronic))

    # Обучение моделей
    mh, lh, cmh, emh, elh, ecmh = [], [], [], [], [], []
    mh, lh, cmh    = epochs(10, 10, model,       dataset, loss_function, optimizer_type, lr=learning_rate)
    emh, elh, ecmh = epochs(10, 10, empty_model, dataset, loss_function, optimizer_type, lr=learning_rate)

    # Запись результатов
    accuracy = 100*numpy.sum(numpy.diagonal(cmh[-1], 0))/numpy.sum(cmh[-1])
    empty_accuracy = 100*numpy.sum(numpy.diagonal(ecmh[-1], 0))/numpy.sum(ecmh[-1])
    trial.set_user_attr('accuracy',       accuracy)
    trial.set_user_attr('empty_accuracy', empty_accuracy)
    print(f"Точности модели и пустой модели: {accuracy} | {empty_accuracy}")

    # Запись истории ошибок
    lh = numpy.concatenate(lh).tolist()
    trial.set_user_attr('loss_history', json.dumps(lh))
    
    return accuracy

# optuna.delete_study(study_name="ID2NN", storage="sqlite:///ID2NN.db")
study = optuna.create_study(study_name="ID2NN", storage="sqlite:///ID2NN.db", direction='maximize', load_if_exists=True)

[I 2024-08-16 09:39:04,826] Using an existing study with name 'ID2NN' instead of creating a new one.


# Оптимизация

In [ ]:
study.optimize(objective, n_trials=3)

In [9]:
dataframe:pandas.DataFrame = study.trials_dataframe()
dataframe[['user_attrs_accuracy', 'user_attrs_empty_accuracy']]

,user_attrs_accuracy,user_attrs_empty_accuracy
0,9.218227,42.755473
1,42.755473,42.755473
2,NaN,NaN


In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)

In [ ]:
study.optimize(objective, n_trials=3)